Import required libraries.

In [2]:
import pydub
import numpy as np
import sounddevice as sd
from matplotlib import pyplot as plt
from scipy.io.wavfile import read, write

D:\anaconda3\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


These are my own implemented functions. 'dotted' outputs 1.5 times the duration, and getDurations outputs the amount of samples in an array so as to be used later.
Indices are:
    Whole note = 0
    Dotted whole note = 1
    Half note = 2
    Dotted half note = 3
    Quarter note = 4
    Dotted quarter note = 5
    etc.

In [3]:
def getDurations(tempo, fs):
    whole_note = 4*60*fs/tempo
    durs = np.array([[whole_note/(2**i),dotted(whole_note/(2**i))] for i in range(6)])
    
    return durs.flatten()

def dotted(dur):
    return dur*1.5

These are the functions from helper.ipynb but edited so as to implement tempo into the mix.

In [4]:
def sin_wave(f, n, fs):
    x = np.linspace(0, 2*np.pi, n)
    xp = np.linspace(0, -1*(n*ring/fs), n)
    y = np.sin(x*f*(n/fs))*np.exp(xp)
    z = np.zeros([n, 2])
    z[:, 0] = y
    z[:, 1] = y
    return z

def play_note(note_id, octave, dur, fs):
    if (note_id < 3) :
        octave += 1
    y = sin_wave(notes_base[note_id]*2**octave, int(notes_duration[dur]), fs)
    sd.play(y, fs)
    sd.wait()
    return 

def put_note(note_id, octave, dur, fs):
    if (note_id < 3) :
        octave += 1
    y = sin_wave(notes_base[note_id]*2**octave, int(notes_duration[dur]), fs)
    return y

def get_music(music_notes, fs):
    m = []
    for item in music_notes:
        y = put_note(item[0], item[1], item[2], fs)
        m.append(y)
    m = np.concatenate(m, 0)
    return m

This is a testbench. We set A4 = 440 Hz and the tempo and ringing values for the sine wave, get the note duration array from the tempo and then proceed to play the riff and the C Major scale.

In [5]:
notes_base = 2**(np.arange(12)/12)*27.5 # A0; A4= 440 Hz
tempo = 150
ring = 7 # how long the sine wave rings. lower value = more ringing/sustain
fs = 44100
notes_duration = getDurations(tempo,fs)
"""
    Whole note = 0
    Dotted whole note = 1
    Half note = 2
    Dotted half note = 3
    Quarter note = 4
    Dotted quarter note = 5
    etc.
"""
notes_ann = ['A', 'A#', 'B', 'C', 'C#', 'D', 'Eb', 'E', 'F', 'F#', 'G', 'G#']

Scale = [[3,4,4], [5,4,4], [7,4,4], [8,4,4], [10,4,4], [0,4,4], [2,4,4], [3,5,4], 
        [2,4,4], [0,4,4], [10,4,4], [8,4,4], [7,4,4], [5,4,4], [3,4,4]]
theRiff = [ [5,4,6], [7,4,6], [8,4,6], [10,4,6], [7,4,4], [3,4,6], [5,4,2] ]

y = get_music(theRiff, fs)
sd.play(y, fs)

Exporting the results to .wav files.

In [6]:
write("the riff.wav", fs, y.astype(np.float32))
write("C Major scale.wav", fs, get_music(theRiff, fs).astype(np.float32))

We now intend on adding reverb using the convultional technique. We first obtain the impulse response of a "Large Hall" and then use the convolution operation to apply it to our signal.

In [8]:
impulse_file_path = 'impulse.wav'
samplerate,impulse_response = read(impulse_file_path)
samplerate

C:\Users\Arad\AppData\Local\Temp\ipykernel_7648\4250220074.py:2: WavFileWarning: Chunk (non-data) not understood, skipping it.
  samplerate,impulse_response = read(impulse_file_path)


48000

As you can see, the sample rate is not 44.1kHz. The project sample rate will be set to this.

In [20]:
notes_base = 2**(np.arange(12)/12)*27.5 # A0; A4= 440 Hz
tempo = 260
ring = 7 # how long the sine wave rings. lower value = more ringing/sustain
fs = samplerate
notes_duration = getDurations(tempo,fs)

jane_maryam = [[0, 4, 4],
 [2, 4, 4],
 [3, 5, 4],
 [5, 5, 4],
 [7, 5, 4],
 [7, 5, 4],
 [7, 5, 4],
 [0, 5, 4],
 [7, 5, 2],
 [7, 5, 4],
 [0, 5, 4],
 [7, 5, 4],
 [0, 5, 4],
 [7, 5, 2],
 [7, 5, 4],
 [8, 5, 4],
 [5, 5, 4],
 [7, 5, 4],
 [3, 5, 2],
 [5, 5, 4],
 [7, 5, 4],
 [3, 5, 4],
 [5, 5, 4],
 [2, 4, 2],
 [3, 5, 4],
 [5, 5, 4],
 [2, 4, 4],
 [3, 5, 4],
 [0, 4, 2],
 [3, 5, 4],
 [2, 4, 4],
 [10, 4, 4],
 [2, 4, 4],
 [10, 4, 2],
 [0, 4, 3],
 [3, 5, 3],
 [3, 5, 4],
 [2, 4, 4],
 [0, 4, 4],
 [2, 4, 4],
 [10, 4, 2],
 [0, 4, 3]]

reverbless = get_music(jane_maryam, fs)
sd.play(reverbless, fs)

In [22]:
left = np.convolve(reverbless[:,0],impulse_response[:,0])
right = np.convolve(reverbless[:,1],impulse_response[:,1])

In [28]:
reverb = np.zeros( (len(left),2))
reverb[:,0] = left
reverb[:,1] = right
reverb /= np.max(np.abs(reverb),axis=0)

In [33]:
sd.play(reverb, fs)
write("Jane maryam with reverb.wav", fs, reverb.astype(np.float32))

We can now put it in a function. Beware that the convolutions will take time.

In [30]:
def reverbify(audio, impulse_response_filepath): # check for fs yourself
    left = np.convolve(audio[:,0],impulse_response[:,0])
    right = np.convolve(audio[:,1],impulse_response[:,1])
    reverb = np.zeros( (len(left),2))
    reverb[:,0] = left
    reverb[:,1] = right
    reverb /= np.max(np.abs(reverb),axis=0) # because of clipping
    return reverb